In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import os, time, json
import pandas as pd
from openai import OpenAI
from anthropic import Anthropic
from tqdm.auto import tqdm
import spacy

import sys
sys.path.append("../../")
import os

import logging
from src.utils import logging_utils
from src.utils import env_utils
from src import functional

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}")
logger.info(f"{transformers.__version__=}")

# client = OpenAI(
#     api_key=os.getenv("OPENAI_KEY"),
# )
client = Anthropic(
    api_key=os.getenv("CLAUDE_KEY"),
)

MODEL_NAME = "claude-3-5-sonnet-20240620"

2024-09-05 16:26:57 __main__ INFO     torch.__version__='2.3.1', torch.version.cuda='12.1'
2024-09-05 16:26:57 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=1, torch.cuda.get_device_name()='NVIDIA RTX A6000'
2024-09-05 16:26:57 __main__ INFO     transformers.__version__='4.44.2'
2024-09-05 16:26:57 httpx DEBUG    load_ssl_context verify=True cert=None trust_env=True http2=False
2024-09-05 16:26:57 httpx DEBUG    load_verify_locations cafile='/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/certifi/cacert.pem'


In [5]:
from src.utils.env_utils import CLAUDE_CACHE_DIR
CLAUDE_CACHE_DIR

'/home/local_arnab/Codes/retrieval/data/LLM_CACHE/claude_cache'

In [6]:
# movie,actor,character
prompt = """
Give me a list of movies where an actor potrayed a famous person. It should be in csv format. Give me as many examples as you can.

movie,actor,character
A Beautiful Mind,Russell Crowe,John Nash
Gandhi,Ben Kingsley,Mahatma Gandhi
"""

In [7]:
# # superhero,similar powers, characteristics

# prompt = """
# Give me a list of superpowered characters with similar powers and traits. The character can be from any universe (Marvel, DC, different anime etc). It should be in csv format. Give me as many examples as you can.

# connection,character1,character2
# kryptonian,Superman,Supergirl
# speedster,Flash,Quicksilver
# billionaire,Batman,Ironman
# blind, Daredevil, Toph Beifong
# """

In [8]:
# # architect,famous buildings

# prompt = """
# Give me a list of 2 famous buildings designed by the same architect or architectural firm. It should be in csv format. Give me as many examples as you can (atleast 20).
# It should be in csv format. Give me as many examples as you can.

# architect,building1,building2
# Frank Lloyd Wright,Fallingwater,Guggenheim Museum
# Zaha Hadid,Heydar Aliyev Center,Guangzhou Opera House
# """

In [9]:
# # sports, famous players

# prompt = """
# Give me a list of 2 famous players who played the same sport. It should be in csv format. Give me as many examples as you can (atleast 20).

# sport,player1,player2
# basketball,Michael Jordan,LeBron James
# tennis,Roger Federer,Rafael Nadal
# Cricket,Michael Vaughan,Sourav Ganguly
# """

In [13]:
# response = client.chat.completions.create(
#     model=MODEL_NAME,
#     messages=[
#         {"role": "system", "content": "You are a helpful assistant."},
#         {"role": "user", "content": prompt},
#     ],
#     temperature=0,
#     max_tokens=4000,
# )
# print(response.choices[0].message.content)

response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1000,
    temperature=0,
    system="You are a helpful assistant.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                }
            ]
        }
    ]
)
print(response.content[0].text)

2024-09-05 16:10:14 anthropic._base_client DEBUG    Request options: {'method': 'post', 'url': '/v1/messages', 'timeout': 600, 'files': None, 'json_data': {'max_tokens': 1000, 'messages': [{'role': 'user', 'content': [{'type': 'text', 'text': '\nGive me a list of movies where an actor potrayed a famous person. It should be in csv format. Give me as many examples as you can.\n\nmovie,actor,character\nA Beautiful Mind,Russell Crowe,John Nash\nGandhi,Ben Kingsley,Mahatma Gandhi\n'}]}], 'model': 'claude-3-5-sonnet-20240620', 'system': 'You are a helpful assistant.', 'temperature': 0}}
2024-09-05 16:10:14 anthropic._base_client DEBUG    Sending HTTP Request: POST https://api.anthropic.com/v1/messages
2024-09-05 16:10:14 httpcore.connection DEBUG    close.started
2024-09-05 16:10:14 httpcore.connection DEBUG    close.complete
2024-09-05 16:10:14 httpcore.connection DEBUG    connect_tcp.started host='api.anthropic.com' port=443 local_address=None timeout=600 socket_options=None
2024-09-05 16:

In [9]:
from src.functional import ask_claude

ask_claude(prompt)

2024-09-05 16:27:56 httpx DEBUG    load_ssl_context verify=True cert=None trust_env=True http2=False
2024-09-05 16:27:56 httpx DEBUG    load_verify_locations cafile='/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/certifi/cacert.pem'
2024-09-05 16:27:56 src.functional DEBUG    found cached gpt4o response for 3b91fa3b8316235a6a4f092036e1ab2c - loading


"Here's an extended list of movies where actors portrayed famous people, in CSV format:\n\nmovie,actor,character\nA Beautiful Mind,Russell Crowe,John Nash\nGandhi,Ben Kingsley,Mahatma Gandhi\nThe Iron Lady,Meryl Streep,Margaret Thatcher\nLincoln,Daniel Day-Lewis,Abraham Lincoln\nRay,Jamie Foxx,Ray Charles\nWalk the Line,Joaquin Phoenix,Johnny Cash\nCapote,Philip Seymour Hoffman,Truman Capote\nThe King's Speech,Colin Firth,King George VI\nFrida,Salma Hayek,Frida Kahlo\nMalcolm X,Denzel Washington,Malcolm X\nThe Theory of Everything,Eddie Redmayne,Stephen Hawking\nBohemian Rhapsody,Rami Malek,Freddie Mercury\nThe Aviator,Leonardo DiCaprio,Howard Hughes\nRocketman,Taron Egerton,Elton John\nAli,Will Smith,Muhammad Ali\nThe Pianist,Adrien Brody,Władysław Szpilman\nSchindler's List,Liam Neeson,Oskar Schindler\nThe Social Network,Jesse Eisenberg,Mark Zuckerberg\nSelena,Jennifer Lopez,Selena Quintanilla-Pérez\nThe Queen,Helen Mirren,Queen Elizabeth II\nJudy,Renée Zellweger,Judy Garland\nChapli